In [1]:
import pickle
import numpy as np
import pandas as pd

In [2]:
with open('df_scaled.pickle','rb') as f:
    df_scaled = pickle.load(f)


In [3]:
df_scaled['Food']

,종가,등락률,거래변동량,코스피지수_증감량
일자,,,,
2010-05-14,0.074920,0.491660,0.000000,0.511587
2010-05-17,0.077067,0.468725,0.025468,0.342590
2010-05-18,0.101557,0.554667,0.041343,0.479910
2010-05-19,0.087859,0.434181,0.027336,0.461179
2010-05-20,0.089447,0.486699,0.028428,0.397058
...,...,...,...,...
2022-07-06,0.417075,0.499675,0.023779,0.320948
2022-07-07,0.417794,0.515987,0.040026,0.673459
2022-07-08,0.410821,0.472507,0.022978,0.574176


# train / test 분할

In [4]:
from sklearn.model_selection import train_test_split
X_train_dic = {}
X_test_dic={}
y_train_dic = {}
y_test_dic = {}
#모델이 각 업종마다 존재한다. 18개
for sector_name, df in df_scaled.items():
    X_train, X_test, y_train, y_test = train_test_split(df.drop('종가', 1), df['종가'], test_size=0.2, random_state=0, shuffle=False)
    X_train_dic[sector_name] = X_train
    X_test_dic[sector_name] = X_test
    y_train_dic[sector_name] = y_train
    y_test_dic[sector_name] = y_test
    
    

C:\Users\JSWonner\AppData\Local\Temp/ipykernel_12656/2295906184.py:8: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  X_train, X_test, y_train, y_test = train_test_split(df.drop('종가', 1), df['종가'], test_size=0.2, random_state=0, shuffle=False)


In [5]:
X_train_dic['Food'].shape, y_train_dic['Food'].shape

((2400, 3), (2400,))

In [6]:
X_test_dic['Food'].shape, y_test_dic['Food'].shape

((600, 3), (600,))

# TensroFlow Dataset을 활용한 시퀀스 데이터셋 구성

In [7]:
import tensorflow as tf

In [8]:
def windowed_dataset(series, window_size, batch_size, shuffle):
    # 텐서플로우 입력은 3D 텐서를 입력으로 받는다.
    # 2차원 데이터를 -> 3차원 (마지막 축 하나 추가)
    series = tf.expand_dims(series, axis=-1)
    
    # numpy array x(input)를 Dataset으로 변환
    ds = tf.data.Dataset.from_tensor_slices(series)
    
    #
    ds = ds.window(window_size + 1, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda w: w.batch(window_size + 1))
    if shuffle:
        ds = ds.shuffle(1000)
    ds = ds.map(lambda w: (w[:-1], w[-1]))
    return ds.batch(batch_size).prefetch(1)

## Hyperparameter를 정의합니다.

In [9]:
WINDOW_SIZE=20
BATCH_SIZE=32

In [10]:
train_dic= {}
test_dic={}

In [12]:
for sector_name, y_train in y_train_dic.items():
    train_data = windowed_dataset(y_train, WINDOW_SIZE, BATCH_SIZE, True)
    train_dic[sector_name] = train_data
    
for sector_name, y_test in y_test_dic.items():
    test_data = windowed_dataset(y_test, WINDOW_SIZE, BATCH_SIZE, False)
    test_dic[sector_name] = test_data

In [14]:
# 아래의 코드로 데이터셋의 구성을 확인해 볼 수 있습니다.
# X: (batch_size, window_size, feature)
# Y: (batch_size, feature)
for data in train_dic['Food'].take(1):
    print(f'데이터셋(X) 구성(batch_size, window_size, feature갯수): {data[0].shape}')
    print(f'데이터셋(Y) 구성(batch_size, window_size, feature갯수): {data[1].shape}')

데이터셋(X) 구성(batch_size, window_size, feature갯수): (32, 20, 1)
데이터셋(Y) 구성(batch_size, window_size, feature갯수): (32, 1)


## 모델

In [15]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Conv1D, Lambda
from tensorflow.keras.losses import Huber
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


model = Sequential([
    # 1차원 feature map 생성
    Conv1D(filters=32, kernel_size=5,
           padding="causal",
           activation="relu",
           input_shape=[WINDOW_SIZE, 1]),
    # LSTM
    LSTM(16, activation='tanh'),
    Dense(16, activation="relu"),
    Dense(1),
])

In [16]:
# Sequence 학습에 비교적 좋은 퍼포먼스를 내는 Huber()를 사용합니다.
loss = Huber()
optimizer = Adam(0.0005)
model.compile(loss=Huber(), optimizer=optimizer, metrics=['mse'])

In [21]:
import os
# earlystopping은 10번 epoch통안 val_loss 개선이 없다면 학습을 멈춥니다.
earlystopping = EarlyStopping(monitor='val_loss', patience=10)

# val_loss 기준 체크포인터도 생성합니다.
#filename = os.path.join('tmp', 'ckeckpointer.ckpt')
mc = {}
for sector_name in train_dic:
    mc[sector_name] = ModelCheckpoint(sector_name+'_model.h5', 
                         save_weights_only=True, 
                         save_best_only=True, 
                         monitor='val_loss', 
                         verbose=1)

In [22]:
history_dic={}
for sector_name, train_data in train_dic.items():
    test_data = test_dic[sector_name]
    history = model.fit(train_data, 
                        validation_data=(test_data), 
                        epochs=50, 
                        callbacks=[mc[sector_name], earlystopping])
    history_dic[sector_name] = history

Epoch 1/50
75/75 [==============================] - 3s 31ms/step - loss: 1.7282e-04 - mse: 3.4564e-04 - val_loss: 2.0416e-04 - val_mse: 4.0831e-04

Epoch 00001: val_loss improved from inf to 0.00020, saving model to Food_model.h5
Epoch 2/50
75/75 [==============================] - 2s 29ms/step - loss: 1.8031e-04 - mse: 3.6062e-04 - val_loss: 2.2993e-04 - val_mse: 4.5986e-04

Epoch 00002: val_loss did not improve from 0.00020
Epoch 3/50
75/75 [==============================] - 2s 28ms/step - loss: 1.7218e-04 - mse: 3.4436e-04 - val_loss: 2.0413e-04 - val_mse: 4.0825e-04

Epoch 00003: val_loss improved from 0.00020 to 0.00020, saving model to Food_model.h5
Epoch 4/50
75/75 [==============================] - 2s 27ms/step - loss: 1.6392e-04 - mse: 3.2785e-04 - val_loss: 1.9545e-04 - val_mse: 3.9090e-04

Epoch 00004: val_loss improved from 0.00020 to 0.00020, saving model to Food_model.h5
Epoch 5/50
75/75 [==============================] - 2s 27ms/step - loss: 1.6150e-04 - mse: 3.2300e-04 -

75/75 [==============================] - 2s 23ms/step - loss: 1.3233e-04 - mse: 2.6466e-04 - val_loss: 1.6261e-04 - val_mse: 3.2522e-04

Epoch 00038: val_loss did not improve from 0.00015
Epoch 39/50
75/75 [==============================] - 2s 23ms/step - loss: 1.3266e-04 - mse: 2.6531e-04 - val_loss: 1.4390e-04 - val_mse: 2.8780e-04

Epoch 00039: val_loss improved from 0.00015 to 0.00014, saving model to Food_model.h5
Epoch 40/50
75/75 [==============================] - 2s 24ms/step - loss: 1.3515e-04 - mse: 2.7030e-04 - val_loss: 1.6642e-04 - val_mse: 3.3283e-04

Epoch 00040: val_loss did not improve from 0.00014
Epoch 41/50
75/75 [==============================] - 2s 25ms/step - loss: 1.3662e-04 - mse: 2.7325e-04 - val_loss: 1.4216e-04 - val_mse: 2.8432e-04

Epoch 00041: val_loss improved from 0.00014 to 0.00014, saving model to Food_model.h5
Epoch 42/50
75/75 [==============================] - 2s 23ms/step - loss: 1.3061e-04 - mse: 2.6122e-04 - val_loss: 1.5012e-04 - val_mse: 3.002

75/75 [==============================] - 2s 24ms/step - loss: 6.3600e-05 - mse: 1.2720e-04 - val_loss: 1.5765e-04 - val_mse: 3.1530e-04

Epoch 00003: val_loss did not improve from 0.00015
Epoch 4/50
75/75 [==============================] - 2s 24ms/step - loss: 6.2077e-05 - mse: 1.2415e-04 - val_loss: 1.7350e-04 - val_mse: 3.4700e-04 loss: 4.4235e-05 - 

Epoch 00004: val_loss did not improve from 0.00015
Epoch 5/50
75/75 [==============================] - 2s 24ms/step - loss: 5.8561e-05 - mse: 1.1712e-04 - val_loss: 1.6140e-04 - val_mse: 3.2279e-04

Epoch 00005: val_loss did not improve from 0.00015
Epoch 6/50
75/75 [==============================] - 2s 24ms/step - loss: 6.6199e-05 - mse: 1.3240e-04 - val_loss: 1.5345e-04 - val_mse: 3.0691e-04

Epoch 00006: val_loss did not improve from 0.00015
Epoch 7/50
75/75 [==============================] - 2s 27ms/step - loss: 7.4544e-05 - mse: 1.4909e-04 - val_loss: 1.9957e-04 - val_mse: 3.9915e-04

Epoch 00007: val_loss did not improve from 0.00


Epoch 00008: val_loss did not improve from 0.00008
Epoch 9/50
75/75 [==============================] - 2s 29ms/step - loss: 8.3809e-05 - mse: 1.6762e-04 - val_loss: 8.1041e-05 - val_mse: 1.6208e-04

Epoch 00009: val_loss improved from 0.00008 to 0.00008, saving model to Metal_model.h5
Epoch 10/50
75/75 [==============================] - 3s 30ms/step - loss: 9.1046e-05 - mse: 1.8209e-04 - val_loss: 8.2700e-05 - val_mse: 1.6540e-04

Epoch 00010: val_loss did not improve from 0.00008
Epoch 11/50
75/75 [==============================] - 3s 36ms/step - loss: 8.4327e-05 - mse: 1.6865e-04 - val_loss: 8.2162e-05 - val_mse: 1.6432e-04

Epoch 00011: val_loss did not improve from 0.00008
Epoch 12/50
75/75 [==============================] - 2s 28ms/step - loss: 9.2647e-05 - mse: 1.8529e-04 - val_loss: 9.2634e-05 - val_mse: 1.8527e-04

Epoch 00012: val_loss did not improve from 0.00008
Epoch 13/50
75/75 [==============================] - 2s 28ms/step - loss: 8.7153e-05 - mse: 1.7431e-04 - val_loss


Epoch 00002: val_loss did not improve from 0.00006
Epoch 3/50
75/75 [==============================] - 2s 25ms/step - loss: 1.2939e-04 - mse: 2.5878e-04 - val_loss: 5.8575e-05 - val_mse: 1.1715e-04

Epoch 00003: val_loss improved from 0.00006 to 0.00006, saving model to Construction_model.h5
Epoch 4/50
75/75 [==============================] - 2s 24ms/step - loss: 1.2983e-04 - mse: 2.5965e-04 - val_loss: 6.5354e-05 - val_mse: 1.3071e-04

Epoch 00004: val_loss did not improve from 0.00006
Epoch 5/50
75/75 [==============================] - 2s 25ms/step - loss: 1.2651e-04 - mse: 2.5302e-04 - val_loss: 6.2128e-05 - val_mse: 1.2426e-04

Epoch 00005: val_loss did not improve from 0.00006
Epoch 6/50
75/75 [==============================] - 2s 25ms/step - loss: 1.3731e-04 - mse: 2.7462e-04 - val_loss: 7.3369e-05 - val_mse: 1.4674e-04

Epoch 00006: val_loss did not improve from 0.00006
Epoch 7/50
75/75 [==============================] - 2s 24ms/step - loss: 1.2909e-04 - mse: 2.5817e-04 - val_l


Epoch 00015: val_loss did not improve from 0.00005
Epoch 16/50
75/75 [==============================] - 2s 24ms/step - loss: 1.0368e-04 - mse: 2.0736e-04 - val_loss: 5.2969e-05 - val_mse: 1.0594e-04

Epoch 00016: val_loss did not improve from 0.00005
Epoch 17/50
75/75 [==============================] - 2s 24ms/step - loss: 1.0431e-04 - mse: 2.0863e-04 - val_loss: 4.6114e-05 - val_mse: 9.2228e-05- loss: 1.2021e-04 - ms

Epoch 00017: val_loss did not improve from 0.00005
Epoch 18/50
75/75 [==============================] - 2s 27ms/step - loss: 1.0527e-04 - mse: 2.1054e-04 - val_loss: 4.8613e-05 - val_mse: 9.7225e-05

Epoch 00018: val_loss did not improve from 0.00005
Epoch 19/50
75/75 [==============================] - 2s 23ms/step - loss: 1.0427e-04 - mse: 2.0853e-04 - val_loss: 4.6171e-05 - val_mse: 9.2342e-05

Epoch 00019: val_loss did not improve from 0.00005
Epoch 20/50
75/75 [==============================] - 2s 29ms/step - loss: 1.0519e-04 - mse: 2.1039e-04 - val_loss: 4.7569e-05

75/75 [==============================] - 2s 27ms/step - loss: 5.1855e-05 - mse: 1.0371e-04 - val_loss: 1.3687e-04 - val_mse: 2.7375e-04

Epoch 00023: val_loss did not improve from 0.00013
Epoch 24/50
75/75 [==============================] - 2s 26ms/step - loss: 5.0814e-05 - mse: 1.0163e-04 - val_loss: 1.3530e-04 - val_mse: 2.7059e-04

Epoch 00024: val_loss did not improve from 0.00013
Epoch 25/50
75/75 [==============================] - 2s 25ms/step - loss: 5.2610e-05 - mse: 1.0522e-04 - val_loss: 1.7056e-04 - val_mse: 3.4112e-04

Epoch 00025: val_loss did not improve from 0.00013
Epoch 1/50
75/75 [==============================] - 2s 27ms/step - loss: 1.4436e-04 - mse: 2.8873e-04 - val_loss: 2.9846e-04 - val_mse: 5.9691e-04

Epoch 00001: val_loss improved from inf to 0.00030, saving model to Tele_model.h5
Epoch 2/50
75/75 [==============================] - 2s 25ms/step - loss: 1.4345e-04 - mse: 2.8690e-04 - val_loss: 2.7145e-04 - val_mse: 5.4290e-04

Epoch 00002: val_loss improved fro

75/75 [==============================] - 2s 27ms/step - loss: 1.6190e-04 - mse: 3.2379e-04 - val_loss: 3.5029e-04 - val_mse: 7.0057e-04

Epoch 00012: val_loss did not improve from 0.00032
Epoch 13/50
75/75 [==============================] - 2s 25ms/step - loss: 1.5970e-04 - mse: 3.1941e-04 - val_loss: 3.2615e-04 - val_mse: 6.5230e-04

Epoch 00013: val_loss did not improve from 0.00032
Epoch 14/50
75/75 [==============================] - 2s 24ms/step - loss: 1.6591e-04 - mse: 3.3183e-04 - val_loss: 3.8678e-04 - val_mse: 7.7355e-04

Epoch 00014: val_loss did not improve from 0.00032
Epoch 15/50
75/75 [==============================] - 2s 29ms/step - loss: 1.6000e-04 - mse: 3.2001e-04 - val_loss: 3.2632e-04 - val_mse: 6.5265e-04

Epoch 00015: val_loss did not improve from 0.00032
Epoch 16/50
75/75 [==============================] - 2s 29ms/step - loss: 1.6008e-04 - mse: 3.2017e-04 - val_loss: 3.3930e-04 - val_mse: 6.7861e-04

Epoch 00016: val_loss did not improve from 0.00032
Epoch 17/50


75/75 [==============================] - 2s 26ms/step - loss: 3.0761e-05 - mse: 6.1522e-05 - val_loss: 1.6894e-04 - val_mse: 3.3788e-04

Epoch 00018: val_loss did not improve from 0.00017
Epoch 19/50
75/75 [==============================] - 2s 25ms/step - loss: 3.1276e-05 - mse: 6.2552e-05 - val_loss: 1.6877e-04 - val_mse: 3.3755e-04

Epoch 00019: val_loss improved from 0.00017 to 0.00017, saving model to Service_model.h5
Epoch 20/50
75/75 [==============================] - 2s 25ms/step - loss: 3.1555e-05 - mse: 6.3111e-05 - val_loss: 1.6938e-04 - val_mse: 3.3876e-04

Epoch 00020: val_loss did not improve from 0.00017
Epoch 21/50
75/75 [==============================] - 2s 26ms/step - loss: 3.1353e-05 - mse: 6.2706e-05 - val_loss: 1.7397e-04 - val_mse: 3.4793e-04

Epoch 00021: val_loss did not improve from 0.00017
Epoch 22/50
75/75 [==============================] - 2s 23ms/step - loss: 3.1576e-05 - mse: 6.3152e-05 - val_loss: 2.1392e-04 - val_mse: 4.2784e-04

Epoch 00022: val_loss did